# EECS759P Coursework 2
- Name: Bheki Maenetja
- Student ID: 230382466

## Imports

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, deque

## Plotting Functions

## Loading Data

In [2]:
# Function provided in undirected_map.py
def load_data(df):
    station_dict = defaultdict(list)
    zone_dict = defaultdict(set)

    # get data row by row
    for index, row in df.iterrows():
        start_station = row[0]
        end_station = row[1]

        line = row[2] 
        
        act_cost = int(row[3])
        
        zone1 = row[4]
        zone2 = row[5]

        # station dictionary of child station tuples (child_name, cost from parent to the child)
        # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
        station_list = station_dict[start_station]
        station_list.append((end_station, line, act_cost))

        # the following two lines add the other direction of the tube "step"
        station_list = station_dict[end_station]
        station_list.append((start_station, line, act_cost))

        # we add the main zone
        zone_dict[start_station].add(zone1)
        # we add the secondary zone

        if zone2 != "0":
            zone_dict[start_station].add(zone2)
            # if the secondary zone is not 0 it's the main zone for the ending station
            zone_dict[end_station].add(zone2)
        else:
            # otherwise the main zone for the ending station is the same as for the starting station
            zone_dict[end_station].add(zone1)

    return station_dict, zone_dict

In [3]:
tube_df = pd.read_csv('tubedata.csv', header=None)
tube_df.head()

,0,1,2,3,4,5
0,Harrow & Wealdstone,Kenton,Bakerloo,3,5,0
1,Kenton,South Kenton,Bakerloo,2,4,0
2,South Kenton,North Wembley,Bakerloo,2,4,0
3,North Wembley,Wembley Central,Bakerloo,2,4,0
4,Wembley Central,Stonebridge Park,Bakerloo,3,4,0


In [4]:
stations, zones = load_data(tube_df)

In [5]:
set(tube_df[0])

{'Acton Town',
 'Aldgate',
 'Aldgate East',
 'Alperton',
 'Angel',
 'Archway',
 'Arnos Grove',
 'Arsenal',
 'Baker Street',
 'Balham',
 'Bank/Monument',
 'Barbican',
 'Barking',
 'Barkingside',
 'Barons Court',
 'Bayswater',
 'Becontree',
 'Belsize Park',
 'Bermondsey',
 'Bethnal Green',
 'Blackfriars',
 'Blackhorse Road',
 'Bond Street',
 'Borough',
 'Boston Manor',
 'Bounds Green',
 'Bow Road',
 'Brent Cross',
 'Bromley-by-Bow',
 'Buckhurst Hill',
 'Burnt Oak',
 'Caledonian Road',
 'Camden Town',
 'Canada Water',
 'Canary Wharf',
 'Canning Town',
 'Cannon Street',
 'Canons Park',
 'Chalfont & Latimer',
 'Chalk Farm',
 'Chancery Lane',
 'Charing Cross',
 'Chesham',
 'Chigwell',
 'Chiswick Park',
 'Chorleywood',
 'Clapham Common',
 'Clapham North',
 'Clapham South',
 'Cockfosters',
 'Colindale',
 'Colliers Wood',
 'Covent Garden',
 'Croxley',
 'Dagenham East',
 'Dagenham Heathway',
 'Debden',
 'Dollis Hill',
 'Ealing Broadway',
 'Ealing Common',
 "Earls' Court",
 'East Acton',
 'East F

In [6]:
stations

defaultdict(list,
            {'Harrow & Wealdstone': [('Kenton', 'Bakerloo', 3)],
             'Kenton': [('Harrow & Wealdstone', 'Bakerloo', 3),
              ('South Kenton', 'Bakerloo', 2)],
             'South Kenton': [('Kenton', 'Bakerloo', 2),
              ('North Wembley', 'Bakerloo', 2)],
             'North Wembley': [('South Kenton', 'Bakerloo', 2),
              ('Wembley Central', 'Bakerloo', 2)],
             'Wembley Central': [('North Wembley', 'Bakerloo', 2),
              ('Stonebridge Park', 'Bakerloo', 3)],
             'Stonebridge Park': [('Wembley Central', 'Bakerloo', 3),
              ('Harlesden', 'Bakerloo', 2)],
             'Harlesden': [('Stonebridge Park', 'Bakerloo', 2),
              ('Willesden Junction', 'Bakerloo', 2)],
             'Willesden Junction': [('Harlesden', 'Bakerloo', 2),
              ('Kensal Green', 'Bakerloo', 3)],
             'Kensal Green': [('Willesden Junction', 'Bakerloo', 3),
              ("Queen's Park", 'Bakerloo', 3)],
 

In [7]:
zones

defaultdict(set,
            {'Harrow & Wealdstone': {'5'},
             'Kenton': {'4', '5'},
             'South Kenton': {'4'},
             'North Wembley': {'4'},
             'Wembley Central': {'4'},
             'Stonebridge Park': {'3', '4'},
             'Harlesden': {'3'},
             'Willesden Junction': {'3'},
             'Kensal Green': {'2', '3'},
             "Queen's Park": {'2'},
             'Kilburn Park': {'2'},
             'Maida Vale': {'2'},
             'Warwick Avenue': {'2'},
             'Paddington': {'1', '2'},
             'Edgware Road': {'1'},
             'Marylebone': {'1'},
             'Baker Street': {'1', '2'},
             "Regent's Park": {'1'},
             'Oxford Circus': {'1'},
             'Piccadilly Circus': {'1'},
             'Charing Cross': {'1'},
             'Embankment': {'1'},
             'Waterloo': {'1'},
             'Lambeth North': {'1'},
             'Elephant & Castle': {'1', '2'},
             'West Ruislip': {'6'},
 

## 2.1 DFS, BFS and UCS

### Depth-First Search

In [8]:
def dfs_search(start, goal):
    if start == goal:
        print("Start and goal are the same!!!")
        return None
    
    frontier = stations[start].copy()
    explored = [start]
    num_explored = 0

    print(f"Start: {start} —>")
    while frontier:
        node = frontier.pop()
        num_explored += 1
        explored.append(node[0])

        # Goal check
        if node[0] == goal:
            print(f"Goal found!\nNumber of exploration = {num_explored}")
            return node

        print(f"{node} ->")
        # Node expansion
        new_nodes = stations[node[0]]
        for n in new_nodes:
            if n[0] not in explored:
                frontier.append(n)    

In [9]:
dfs_search("Euston", "Victoria")

Start: Euston —>
('Warren Street', 'Victoria', 1) ->
('Oxford Circus', 'Victoria', 2) ->
('Green Park', 'Victoria', 2) ->
Goal found!
Number of exploration = 4


('Victoria', 'Victoria', 2)

### Breadth-First Search

In [10]:
def bfs_search(start, goal):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None
    
    frontier = stations[start].copy()
    explored = [start]
    num_explored = 0

    print(f"Start: {start} —>")
    while frontier:
        node = frontier.pop(0)
        num_explored += 1
        explored.append(node[0])

        # Goal check
        if node[0] == goal:
            print(f"Goal found!\nNumber of exploration = {num_explored}")
            return node

        print(f"{node} ->")
        # Node expansion
        new_nodes = stations[node[0]]
        for n in new_nodes:
            if n[0] not in explored:
                frontier.append(n) 

In [11]:
bfs_search("Euston", "Victoria")

Start: Euston —>
('Mornington Crescent', 'Northern', 2) ->
('Warren Street', 'Northern', 1) ->
("King's Cross St. Pancras", 'Northern', 2) ->
("King's Cross St. Pancras", 'Victoria', 2) ->
('Warren Street', 'Victoria', 1) ->
('Camden Town', 'Northern', 1) ->
('Goodge Street', 'Northern', 2) ->
('Oxford Circus', 'Victoria', 2) ->
('Euston Square', 'Circle', 2) ->
('Farringdon', 'Circle', 4) ->
('Euston Square', 'Hammersmith & City', 2) ->
('Farringdon', 'Hammersmith & City', 4) ->
('Farringdon', 'Metropolitan', 4) ->
('Euston Square', 'Metropolitan', 2) ->
('Angel', 'Northern', 2) ->
('Caledonian Road', 'Piccadilly', 5) ->
('Russell Square', 'Piccadilly', 2) ->
('Highbury & Islington', 'Victoria', 4) ->
('Euston Square', 'Circle', 2) ->
('Farringdon', 'Circle', 4) ->
('Euston Square', 'Hammersmith & City', 2) ->
('Farringdon', 'Hammersmith & City', 4) ->
('Farringdon', 'Metropolitan', 4) ->
('Euston Square', 'Metropolitan', 2) ->
('Angel', 'Northern', 2) ->
('Caledonian Road', 'Piccadil

('Victoria', 'Victoria', 2)

### Uniform Cost Search

In [19]:
def ucs_search(start, goal):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    frontier = stations[start].copy()
    frontier.sort(key=lambda x: x[2])
    
    explored = [start]
    num_explored = 0

    print(f"Start: {start} —>")
    while frontier:
        node = frontier.pop(0)
        num_explored += 1
        explored.append(node[0])

        # Goal check
        if node[0] == goal:
            print(f"Goal found!\nNumber of exploration = {num_explored}")
            return node
            # return explored

        print(f"{node} ->")
        # Node expansion
        new_nodes = stations[node[0]]
        for n in new_nodes:
            if n[0] not in explored:
                frontier.append(n)

        frontier.sort(key=lambda x: x[2])

In [20]:
ucs_search("Euston", "Victoria")

Start: Euston —>
('Warren Street', 'Northern', 1) ->
('Warren Street', 'Victoria', 1) ->
('Mornington Crescent', 'Northern', 2) ->
('Camden Town', 'Northern', 1) ->
("King's Cross St. Pancras", 'Northern', 2) ->
("King's Cross St. Pancras", 'Victoria', 2) ->
('Goodge Street', 'Northern', 2) ->
('Tottenham Court Road', 'Northern', 1) ->
('Leicester Square', 'Northern', 1) ->
('Covent Garden', 'Piccadilly', 1) ->
('Holborn', 'Piccadilly', 1) ->
('Chancery Lane', 'Central', 1) ->
('Oxford Circus', 'Victoria', 2) ->
('Bond Street', 'Central', 1) ->
('Marble Arch', 'Central', 1) ->
('Goodge Street', 'Northern', 2) ->
('Oxford Circus', 'Victoria', 2) ->
('Kentish Town', 'Northern', 2) ->
('Chalk Farm', 'Northern', 2) ->
('Euston Square', 'Circle', 2) ->
('Euston Square', 'Hammersmith & City', 2) ->
('Euston Square', 'Metropolitan', 2) ->
('Angel', 'Northern', 2) ->
('Russell Square', 'Piccadilly', 2) ->
('Euston Square', 'Circle', 2) ->
('Euston Square', 'Hammersmith & City', 2) ->
('Euston 

('Victoria', 'Victoria', 2)

In [36]:
stations["Euston"]

[('Mornington Crescent', 'Northern', 2),
 ('Warren Street', 'Northern', 1),
 ("King's Cross St. Pancras", 'Northern', 2),
 ("King's Cross St. Pancras", 'Victoria', 2),
 ('Warren Street', 'Victoria', 1)]

In [27]:
my_d = [("abcdefg"[y],y) for y in range(7)]
my_d

[('a', 0), ('b', 1), ('c', 2), ('d', 3), ('e', 4), ('f', 5), ('g', 6)]

In [33]:
my_d.sort(key=lambda x: x[1], reverse=True)
my_d

[('g', 6), ('f', 5), ('e', 4), ('d', 3), ('c', 2), ('b', 1), ('a', 0)]

In [16]:
my_d.copy()

[2, 3]